# Modeling #

## Import APIs ##

In [1]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

import wfdb
import ast

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Load data ##

### Metadata ###

In [3]:
ptbxl_df = pd.read_csv('./cleaned_data/cleaned_ptbxl_metadata.csv', index_col='ecg_id')

In [3]:
ptbxl_df.head()

,age,sex,device,validated_by_human,diagnostic_superclass,strat_fold,filename_lr,filename_hr
ecg_id,,,,,,,,
1,56.0,1,CS-12 E,True,['NORM'],3,records100/00000/00001_lr,records500/00000/00001_hr
2,19.0,0,CS-12 E,True,['NORM'],2,records100/00000/00002_lr,records500/00000/00002_hr
3,37.0,1,CS-12 E,True,['NORM'],5,records100/00000/00003_lr,records500/00000/00003_hr
4,24.0,0,CS-12 E,True,['NORM'],3,records100/00000/00004_lr,records500/00000/00004_hr
5,19.0,1,CS-12 E,True,['NORM'],4,records100/00000/00005_lr,records500/00000/00005_hr


In [4]:
metadata = ptbxl_df.loc[:, ['age', 'sex', 'device', 'validated_by_human']].copy()
metadata.head()

,age,sex,device,validated_by_human
ecg_id,,,,
1,56.0,1,CS-12 E,True
2,19.0,0,CS-12 E,True
3,37.0,1,CS-12 E,True
4,24.0,0,CS-12 E,True
5,19.0,1,CS-12 E,True


### Waveform data ###

In [5]:
waveform_data = []
for idx in ptbxl_df.index:
    record_path = ptbxl_df.loc[idx]['filename_hr']
    waveform_df = pd.read_csv('./cleaned_data/waveform_data/' + record_path + '.csv', index_col='Time (s)')
    waveform_data.append(waveform_df)
waveform_data = np.array(waveform_data)
waveform_data.shape

KeyboardInterrupt: 

In [5]:
waveform_data = np.load("./cleaned_data/waveform_np.npy")

## Create recommended train-test split ##

This recommended train-test split code was obtained from the downloaded folder with the dataset: https://physionet.org/content/ptb-xl/1.0.3/.

In [6]:
# Split data into train and test
test_fold = 10

# Train
waveform_train = waveform_data[np.where(ptbxl_df.strat_fold != test_fold)]
metadata_train = metadata[ptbxl_df.strat_fold != test_fold]
y_train = ptbxl_df[ptbxl_df.strat_fold != test_fold].diagnostic_superclass

# Test
waveform_test = waveform_data[np.where(ptbxl_df.strat_fold == test_fold)]
metadata_test = metadata[ptbxl_df.strat_fold == test_fold]
y_test = ptbxl_df[ptbxl_df.strat_fold == test_fold].diagnostic_superclass

## Normalize waveform data ##

In [7]:
def normalize_waveform(waveform_data):
    # Code generated from Bing Copilot
    normalized_data = np.empty_like(waveform_train)
    for i in range(waveform_data.shape[0]):
        for j in range(waveform_data.shape[2]):
            min_val = np.min(waveform_data[i, :, j])
            max_val = np.max(waveform_data[i, :, j])

            if max_val == min_val:
                normalized_data[i, :, j] = 0
            else:
                normalized_data[i, :, j] = (waveform_data[i, :, j] - min_val) / (max_val - min_val)
    return normalized_data

In [8]:
# Since normalization occurs only within each record, there will be no contamination from train data
normalized_waveform_train = normalize_waveform(waveform_train)
normalized_waveform_test = normalize_waveform(waveform_test)

## Normalize and one-hot encode metadata ##

In [9]:
# Code generated from ChatGPT
scaler_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

encoder_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', scaler_transformer, ['age']),
        ('cat', encoder_transformer, ['sex', 'device', 'validated_by_human'])
    ])

normalized_metadata_train = preprocessor.fit_transform(metadata_train).toarray()
normalized_metadata_test = preprocessor.transform(metadata_test).toarray()

## Initialize Dataloaders ##

In [10]:
waveform_train_set = torch.from_numpy(normalized_waveform_train)
waveform_test_set = torch.from_numpy(normalized_waveform_test)

metadata_train_set = torch.from_numpy(normalized_metadata_train)
metadata_test_set = torch.from_numpy(normalized_metadata_test)

In [11]:
batch_size = 32

train_dataset = TensorDataset(torch.from_numpy(normalized_waveform_train).float(), torch.from_numpy(normalized_metadata_train))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

#waveform_train_loader = DataLoader(waveform_train_set, batch_size=batch_size)
waveform_test_loader = DataLoader(waveform_test_set, batch_size=batch_size)

#metadata_train_loader = DataLoader(metadata_train_set, batch_size=batch_size)
metadata_test_loader = DataLoader(metadata_test_set, batch_size=batch_size)

## CNN autoencoder + LSTM metadata model ##

In [12]:
class CNNAutoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv1d(in_channels=12,
                      out_channels=128,
                      kernel_size=5,
                      stride=1),
            nn.MaxPool1d(kernel_size=2, 
                         stride=2),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(in_channels=128,
                               out_channels=12,
                               kernel_size=5,
                               stride=1),
            nn.Sigmoid(),
            nn.Upsample(size=1000)
        )
    
    def forward(self, x):
        encoded_output = self.encoder(x)
        decoded_output = self.decoder(encoded_output)
        return decoded_output

In [13]:
class MetadataLSTM(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(16, 32, batch_first=True)
        self.output_layer = nn.Linear(32, 16)
    
    def forward(self, x, h):
        lstm_outputs, h_n = self.lstm(x, h)
        outputs = self.output_layer(lstm_outputs)
        return outputs, h_n

In [31]:
class CombinedModel(nn.Module):
    def __init__(self, cnn_autoencoder, lstm_model):
        super().__init__()
        self.cnn_autoencoder = cnn_autoencoder
        self.lstm_model = lstm_model
        #result of combined output is 28x12016
        self.metadata_projection = nn.Linear(16, 128)
        self.conv1 = nn.Conv1d(in_channels=12+128, out_channels=128, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=128, out_channels=12, kernel_size=3, padding=1)
        
    def forward(self, ecg_data, metadata, hc):
        cnn_output = self.cnn_autoencoder(ecg_data)
        lstm_outputs, hc_n = self.lstm_model(metadata, hc)
        projected_metadata = self.metadata_projection(lstm_outputs[:, -1, :])
        

        # Reshape cnn_output and extract the last output from lstm_outputs
        #Reshape cnn_output to (batch_size, channels, sequence_length)
        cnn_output = cnn_output.view(cnn_output.size(0), 12, -1)
        
        projected_metadata = projected_metadata.unsqueeze(2).repeat(1, 1, cnn_output.size(2))
        combined_output = torch.cat((cnn_output, projected_metadata), dim=1)

        
        combined_output = torch.nn.functional.relu(self.conv1(combined_output))
        output = self.conv2(combined_output)

        return output, hc_n

### Training ###

In [32]:
cnn_autoencoder_model = CNNAutoencoder()
lstm_metadata_model = MetadataLSTM()

combinedModel = CombinedModel(cnn_autoencoder_model, lstm_metadata_model)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(combinedModel.parameters(), lr=5e-4)

combinedModel.to(device)

CombinedModel(
  (cnn_autoencoder): CNNAutoencoder(
    (encoder): Sequential(
      (0): Conv1d(12, 128, kernel_size=(5,), stride=(1,))
      (1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (2): ReLU()
    )
    (decoder): Sequential(
      (0): ConvTranspose1d(128, 12, kernel_size=(5,), stride=(1,))
      (1): Sigmoid()
      (2): Upsample(size=1000, mode='nearest')
    )
  )
  (lstm_model): MetadataLSTM(
    (lstm): LSTM(16, 32, batch_first=True)
    (output_layer): Linear(in_features=32, out_features=16, bias=True)
  )
  (metadata_projection): Linear(in_features=16, out_features=128, bias=True)
  (conv1): Conv1d(140, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv2): Conv1d(128, 12, kernel_size=(3,), stride=(1,), padding=(1,))
)

In [33]:
def train_combined(model, train_loader, criterion, optimizer, nepoch=10):
    for epoch in range(nepoch):
        for batch_data, batch_metadata in train_loader:
            ecg_data = batch_data.to(device).permute(0, 2, 1).float()
            batch_size = ecg_data.size(0) 
            batch_metadata = batch_metadata.to(device).float()
            
            #reshape for lstm
            batch_metadata = batch_metadata.view(batch_metadata.size(0), -1, 16)
        
            num_layers = model.lstm_model.lstm.num_layers
            hidden_size = model.lstm_model.lstm.hidden_size
            h_0 = torch.zeros(num_layers, batch_size, hidden_size).to(device)
            c_0 = torch.zeros(num_layers, batch_size, hidden_size).to(device)
            
            optimizer.zero_grad()
            reconstructed_data, (h_n, c_n) = model(ecg_data, batch_metadata, (h_0, c_0))
            loss = criterion(reconstructed_data, ecg_data)
            loss.backward()
            optimizer.step()
        
        print(f"Epoch [{epoch+1}/{nepoch}], Loss: {loss.item():.4f}")
            

In [34]:
# Train the model
train_combined(combinedModel, train_loader, criterion, optimizer)

Epoch [1/10], Loss: 0.0042
Epoch [2/10], Loss: 0.0025
Epoch [3/10], Loss: 0.0016
Epoch [4/10], Loss: 0.0012
Epoch [5/10], Loss: 0.0011
Epoch [6/10], Loss: 0.0012
Epoch [7/10], Loss: 0.0011
Epoch [8/10], Loss: 0.0009
Epoch [9/10], Loss: 0.0009
Epoch [10/10], Loss: 0.0010


In [13]:
torch.save(cnn_autoencoder_model.state_dict(), "./models/cnn_autoencoder.pth")
print(f"Model saved")

Model saved


### Plotting ###

In [ ]:
def plot_waveform(waveform_df, segment_length=1000, start_index=0):
    plt.figure(figsize=(12, 6))
    for col in waveform_df.columns:
        plt.plot(waveform_df.index, waveform_df[col], label=f'Lead {col}')
    
    plt.title('ECG Waveform')
    plt.xlabel('Time (seconds)')
    plt.ylabel('Amplitude')
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.grid(True)
    plt.show()

## TCN Autoencoder ##

Model Card for the Hybrid Autoencoder Model Name: Hybrid Autoencoder for ECG and Metadata

Description: This model is designed to learn compressed representations of combined ECG waveform and patient metadata. It utilizes separate pathways for waveform data and metadata, merging them into a dense representation which is then used to reconstruct both types of data.

Model Architecture:

Waveform Pathway: Convolutional layers followed by pooling and flattening. Metadata Pathway: Dense layers. Combined Encoding and Decoding: Dense layers. Intended Use: Intended for anomaly detection in ECG data where additional patient metadata is available and considered relevant.

Data Used for Training: Assumes a dataset comprising ECG waveform data aligned with patient metadata such as age, sex, and device information.

Limitations: The model's effectiveness is highly dependent on the quality and preprocessing of the input data. The architecture needs fine-tuning and validation using real-world data to ensure robustness.

Ethical Considerations: Care should be taken to avoid biases that may arise from imbalanced data across different demographic groups. Privacy concerns should be addressed when handling patient data.

This framework sets up the foundation of your model; further tuning, training, and validation steps are needed to adapt it to specific tasks or datasets.

In [17]:
num_unique_devices = metadata['device'].nunique()
print(f"Number of unique devices: {num_unique_devices}")

Number of unique devices: 11


In [18]:
from pytorch_tcn import TCN

class TCNAutoencoder(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size, dropout, metadata_dims):
        super(TCNAutoencoder, self).__init__()
        self.encoder = TCN(
            num_inputs=num_inputs,
            num_channels=num_channels,
            kernel_size=kernel_size,
            dropout=dropout,
            causal=True,
        )
        self.age_embedding = nn.Linear(1, metadata_dims[0])  # Age is a single value
        self.sex_embedding = nn.Linear(2, metadata_dims[1])  # Sex is one-hot encoded (2 columns)
        self.device_embedding = nn.Linear(num_unique_devices, metadata_dims[2]) #one hot (11 cols)
        self.validated_embedding = nn.Linear(2, metadata_dims[3]) #one hot (2 cols)
        
        decoder_input_dim = num_channels[-1] + sum(metadata_dims)
        self.decoder = TCN(
            num_inputs=decoder_input_dim,
            num_channels=num_channels[::-1],
            kernel_size=kernel_size,
            dropout=dropout,    
            causal=True,
            output_projection=num_inputs,
        )
        
    def forward(self, x, metadata):
        encoded = self.encoder(x)
        
        age_emb = self.age_embedding(metadata[:, 0].unsqueeze(1))
        sex_emb = self.sex_embedding(metadata[:, 1:3])
        device_emb = self.device_embedding(metadata[:, 3:-2])
        validated_emb = self.validated_embedding(metadata[:, -2:])
        
        metadata_emb = torch.cat([age_emb, sex_emb, device_emb, validated_emb], dim=-1)
        metadata_emb = metadata_emb.unsqueeze(2).expand(-1, -1, encoded.size(2))
        
        concatenated = torch.cat([encoded, metadata_emb], dim=1)
        decoded = self.decoder(concatenated)
        return decoded

In [19]:
batch_size = 32
num_inputs = 12  # Assuming 12 input channels in the ECG data
num_channels = [32, 64, 128]  # Example number of channels in each residual block of the encoder
kernel_size = 3  # Example kernel size for the TCN layers
dropout = 0.2  # Example dropout rate
metadata_dims = [10, 5, 20, 5]  # Example embedding dimensions for age, sex, and device, and validated

num_unique_devices = metadata['device'].nunique()
print(normalized_metadata_train.shape)
assert num_unique_devices == normalized_metadata_train.shape[1] - 5, "Number of unique devices should match the number of device columns in metadata"

model = TCNAutoencoder(num_inputs, num_channels, kernel_size, dropout, metadata_dims)

model.to(device)

train_dataset = TensorDataset(torch.from_numpy(normalized_waveform_train).float(), torch.from_numpy(normalized_metadata_train))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 10

(19601, 16)


In [20]:
for epoch in range(num_epochs):
    for batch_data, batch_metadata in train_loader:
        ecg_data = batch_data.to(device).permute(0, 2, 1).float()
        batch_metadata = batch_metadata.to(device).float()
        
        optimizer.zero_grad()
        reconstructed_data = model(ecg_data, batch_metadata)
        loss = criterion(reconstructed_data, ecg_data)
        loss.backward()
        optimizer.step()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [1/10], Loss: 0.0008
Epoch [2/10], Loss: 0.0001
Epoch [3/10], Loss: 0.0001
Epoch [4/10], Loss: 0.0001
Epoch [5/10], Loss: 0.0001
Epoch [6/10], Loss: 0.0000


KeyboardInterrupt: 

In [13]:
torch.save(model.state_dict(), "./models/tcn.pth")
print(f"Model saved")

Model saved


In [ ]:
#make sure outputs are 1k x 12